### Import packages

In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import load_npz
from scipy import sparse

### Read data

In [2]:
df_val = pd.read_csv('df_val.csv')

In [3]:
R = load_npz('R_train.npz')

In [4]:
n_users,n_movies = R.shape

### Construct accuracy function

In [5]:
one_idxs=[]
minus_one_idxs=[]
hold_out=[]
hold_out_minus=[]

In [6]:
for i in range(n_users):
    one_idxs.append(np.where(R[[i],:].toarray()[0] == 1)[0])
    minus_one_idxs.append(np.where(R[[i],:].toarray()[0] == -1)[0])
    hold_out.append(df_val.query(f"userId=={i} & rating==1").movieId.values)
    hold_out_minus.append(df_val.query(f"userId=={i} & rating==-1").movieId.values)

In [7]:
def accuracy_func(S,k=10):
    accuracy = []
    for i in range(n_users):
        output = S[i]
#         output = output.to('cpu').detach().numpy()[0]
        np.put(output,one_idxs[i],-np.inf)
        np.put(output,minus_one_idxs[i],-np.inf)
        c = len(np.intersect1d(np.argsort(output)[::-1][:k],hold_out[i]))
        nc = len(np.intersect1d(np.argsort(output)[::-1][:k],hold_out_minus[i]))
#         acc = np.max([0,(c-nc)/(np.min([k,len(hold_out[i])+1]))]) ## Recal@K
        acc = np.max([0,(c-nc)/k]) ## HR@K
        accuracy.append(acc)
    return np.mean(accuracy)

### EASE model

In [8]:
def EASE(x,l2=1500):
    G = np.matmul(x.T,x)
    diag_indices = np.diag_indices(G.shape[0])
    G[diag_indices] += l2
    p = np.linalg.inv(G)
    B = p/(-np.diag(p))
    B[diag_indices] = 0
    S = np.matmul(x,B)
    return S

### Training and accuracy

In [9]:
x = np.float32(R.toarray())

In [10]:
S = EASE(x)

In [11]:
accuracy_func(S)

0.33200331125827814